In [21]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "changeme"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ProjectTwoDashboard')
#app.add_external_link = True

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(id='customer-image',height = '100', width = '100', src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    html.Center(html.B(html.H1('Bijal Chauhan Dashboard from CS340'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='rd',
            options=[
                {'label':'Water Rescue','value':'Water Rescue'},
                {'label':'Mountain Rescue','value':'Mountain Rescue'},
                {'label':'Disaster Rescue','value':'Disaster Rescue'},
                {'label':'Reset','value':'Reset'}
            ],
            value='Water Rescue',
            labelStyle={'display':'inline-block'}
        ),
        
     ),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        page_size=10,
        style_table={'height':'400px','overflowY':'auto','overflowX':'auto'},
        style_header={
            'backgroundColor':'rgb(230,230,230)',
            'fontWeight':'bold'        
        },
        style_data={
            'whiteSpace':'normal',
            'height':'auto'
        },
        
        #tooltips that we are going to use on the table so that we know what information we are looking at
        tooltip ={i: {
        'value': i,
        'use_with': 'both'  # both refers to header & data cell
    } for i in df.columns},
        tooltip_delay=0,
        tooltip_duration = None,
        
        #sorting features that we are going to use
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_rows=[],
        selected_columns=[],
        page_action="native",
        page_current=0,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data')],
              [Input('rd','value')])
def update_output(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
       
        if value == 'Water Rescue':
            df = pd.Dataframe.from_records(shelter.read_all({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                         {'$or': [
                                                             {'breed': 'Labrador Retriever Mix'},
                                                             {'breed': 'Chesapeake Bay Retriever'},
                                                             {'breed': 'Newfoundland Mix'}]
                                                         },
                                                         {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                   {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                         }]
                                               }))

        elif value == 'Mountain or Wilderness Rescue':
            df = pd.Dataframe.from_records(shelter.read_all({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                         {'$or': [
                                                             {'breed': 'German Shepherd'},
                                                             {'breed': 'Alaskan Malamute'},
                                                             {'breed': 'Old English Sheepdog'},
                                                             {'breed': 'Rottweiler'}]
                                                         },
                                                         {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                   {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                         }]
                                               }))
            

        elif value == 'Disaster Rescue or Individual Tracking':
            df = pd.Dataframe.from_records(shelter.read_all({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                         {'$or': [
                                                             {'breed': 'Doberman Pinscher'},
                                                             {'breed': 'German Shepherd'},
                                                             {'breed': 'Golden Retriever'},
                                                             {'breed': 'Bloodhound'},
                                                             {'breed': 'Rottweiler'}]
                                                         },
                                                         {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                   {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                         }]
                                               }))
            
        elif value == 'RESET':
            df = pd.DataFrame.from_records(shelter.read_all({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return data




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(            
            figure = px.pie(
               data_frame=dff, 
               values = values, 
               names = names, 
               color_discrete_sequence=px.colors.sequential.RdBu,
               width=600, 
               height=500   
            )
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    
    
    
    # Austin TX is at [30.75,-97.48]
    return [ dl.Map(style={'width': '800px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and pop up
        dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
        dl.Tooltip(dff['breed']),        
        dl.Popup([
        html.H1("Animal Name"),
        html.P(dff.loc[0,'name'])
        ])
        ])
])]


app



                               
   

